In [ ]:
# !pip install -U  boto3 huggingface_hub==0.24.5 mteb==1.6.1

## APPROACH 1

In [ ]:
#https://github.com/aws-samples/amazon-bedrock-samples/blob/main/embeddings/Titan-V2-Embeddings.ipynb

import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(assumed_role: Optional[str] = None, region: Optional[str] = 'us-east-1',runtime: Optional[bool] = True,external_id=None, ep_url=None):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides 
    """
    target_region = region

    print(f"Create new client\n  Using region: {target_region}:external_id={external_id}: ")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        if external_id:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
                ExternalId=external_id
            )
        else:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
            )
        print(f"Using role: {assumed_role} ... sts::successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    if ep_url:
        bedrock_client = session.client(service_name=service_name,config=retry_config,endpoint_url = ep_url, **client_kwargs )
    else:
        bedrock_client = session.client(service_name=service_name,config=retry_config, **client_kwargs )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

In [ ]:
import json
import boto3

class TitanEmbeddings(object):
    accept = "application/json"
    content_type = "application/json"
    
    def __init__(self, model_id="amazon.titan-embed-text-v2:0", boto3_client=None, region_name='us-east-1'):
        
        if boto3_client:
            self.bedrock_boto3 = boto3_client
        else:
            # self.bedrock_boto3 = boto3.client(service_name='bedrock-runtime')
            self.bedrock_boto3 = boto3.client(
                service_name='bedrock-runtime', 
                region_name=region_name, 
            )
        self.model_id = model_id

    def __call__(self, text, dimensions, normalize=True):
        """
        Returns Titan Embeddings

        Args:
            text (str): text to embed
            dimensions (int): Number of output dimensions.
            normalize (bool): Whether to return the normalized embedding or not.

        Return:
            List[float]: Embedding
            
        """

        body = json.dumps({
            "inputText": text,
            "dimensions": dimensions,
            "normalize": normalize
        })

        response = self.bedrock_boto3.invoke_model(
            body=body, modelId=self.model_id, accept=self.accept, contentType=self.content_type
        )

        response_body = json.loads(response.get('body').read())

        return response_body['embedding']

In [ ]:
import json
import os
import sys

import boto3

boto3_bedrock_runtime = get_bedrock_client() #boto3.client('bedrock')

bedrock_embeddings = TitanEmbeddings(model_id="amazon.titan-embed-text-v2:0", boto3_client=boto3_bedrock_runtime)
bedrock_embeddings

In [ ]:
prompt_data = "Amazon Bedrock supports foundation models from industry-leading providers such as \
AI21 Labs, Anthropic, Stability AI, and Amazon. Choose the model that is best suited to achieving \
your unique goals."


modelId = "amazon.titan-embed-text-v2:0"  # 
accept = "application/json"
contentType = "application/json"



sample_model_input={
    "inputText": prompt_data,
    "dimensions": 256,
    "normalize": True
}

body = json.dumps(sample_model_input)

response = boto3_bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

embedding = response_body.get("embedding")
print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

In [ ]:
embedding= bedrock_embeddings(text=prompt_data, dimensions=256, normalize=True)
print(f"The embedding vector has {len(embedding)} values\n{embedding[0:3]+['...']+embedding[-3:]}")

## APPROACH 2

In [ ]:
#https://github.com/aws-samples/amazon-bedrock-samples/blob/main/genai-use-cases/aws-glue-metadata-generation/how_to_generate_metadata_for_glue_data_catalog_w_bedrock.ipynb

In [ ]:
import os, sys, json
import boto3 
from botocore.config import Config
from typing import Optional
import langchain
import logging
from datetime import date, datetime
import pprint 


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    #  create a boto3 session with the specified region and, optionally, an AWS profile name from the `AWS_PROFILE` environment variable.
    if region is None: 
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    # if an `assumed_role` is provided assume that role using STS and retrieve the temporary credential for the client.
    if assumed_role: 
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        print(assumed_role)
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    # create the boto3 client for the `bedrock-runtime` or `bedrock` service, based on the `runtime` flag, with the specified region, credentials (if assumed role is used), and a retry configuration.
    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

In [ ]:
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
embeddings_model_id= "amazon.titan-embed-text-v2:0"

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
GLUE_CRAWLER_ARN = '<YOUR_AWS_GLUE_CRAWLER_IAM_ROLE>'

bedrock_client= get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

test_client = boto3.client("default", region_name="us-east-1")

## APPROACH 3

In [ ]:
import litellm
response = litellm.embedding(
    model="bedrock/amazon.titan-embed-text-v1",
    model_id="provisioned-model-arn",
    input=["hi"],
)

## APPRAOCH 4

In [ ]:
import json
import boto3

# Initialize the AWS Bedrock runtime client
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"  # Ensure this matches your ARN region
)

# Define the model ARN
model_arn = "arn:aws:bedrock:us-east-1:942286715197:application-inference-profile/4eix5vfvs8bmm"
model_arn = "amazon.titan-embed-text-v2:0"
# Define the input text
input_text = "This is a test sentence for embedding generation."

# Construct the payload (modify as needed based on model requirements)
payload = {
    "inputText": input_text
}

# Invoke the embedding model
response = bedrock_runtime.invoke_model(
    modelId=model_arn, 
    contentType="application/json",
    body=json.dumps(payload)
)

# Parse and print the response
response_body = json.loads(response["body"].read())
print(response_body)


## APPROACH 5 

In [ ]:
import boto3
import json

def get_embeddings_with_profile(text, profile_arn, region="us-east-1"):
    """
    Get embeddings using a Bedrock application inference profile ARN.
    
    Args:
        text (str): The input text to embed
        profile_arn (str): The ARN of the Bedrock application inference profile
        region (str): AWS region where Bedrock is available
        
    Returns:
        list: The embedding vector
    """
    # Create a boto3 session for the specified region
    session = boto3.Session(region_name=region)
    
    # Create a Bedrock Runtime client
    bedrock_runtime = session.client(
        service_name='bedrock-runtime',
        region_name=region
    )
    
    # Prepare the request body
    request_body = {
        "inputText": text
    }
    
    # Convert the request body to bytes
    body = json.dumps(request_body).encode('utf-8')
    
    # Invoke the model using the application inference profile
    response = bedrock_runtime.invoke_model_with_application_inference_profile(
        applicationInferenceProfileArn=profile_arn,
        body=body
    )
    
    # Parse the response
    response_body = json.loads(response['body'].read())
    embedding = response_body.get('embedding')
    
    return embedding

# Example usage
if __name__ == "__main__":
    # Your specific application inference profile ARN
    profile_arn = "arn:aws:bedrock:us-east-1:942286715197:application-inference-profile/4eix5vfvs8bmm"
    region = "us-east-1"
    
    text = "This is a sample text to embed."
    embedding = get_embeddings_with_profile(text, profile_arn, region)
    
    # Print embedding dimensions
    print(f"Embedding dimension: {len(embedding)}")

## APPRAOCH 7

In [ ]:
from llama_index.embeddings import BedrockEmbedding
from llama_index.core import Document, VectorStoreIndex
import boto3
import json

class BedrockProfileEmbedding:
    def __init__(self, profile_arn, region="us-east-1"):
        # Initialize boto3 client
        session = boto3.Session(region_name=region)
        self.client = session.client('bedrock-runtime')
        self.profile_arn = profile_arn
        self.dimension = None  # Will be set after first embedding
        
    def _get_embedding(self, text):
        request_body = {"inputText": text}
        body = json.dumps(request_body).encode('utf-8')
        
        response = self.client.invoke_model_with_application_inference_profile(
            applicationInferenceProfileArn=self.profile_arn,
            body=body
        )
        
        response_body = json.loads(response['body'].read())
        embedding = response_body.get('embedding')
        
        # Set dimension if not already set
        if self.dimension is None and embedding:
            self.dimension = len(embedding)
            
        return embedding
    
    def get_text_embedding(self, text):
        return self._get_embedding(text)
        
    def get_query_embedding(self, query):
        return self._get_embedding(query)
    
    # Required methods for LlamaIndex compatibility
    def embed_documents(self, documents):
        return [self.get_text_embedding(doc.text) for doc in documents]
    
    def embed_query(self, query):
        return self.get_query_embedding(query)

# Example usage with LlamaIndex
def index_documents_with_bedrock():
    # Your application inference profile ARN
    profile_arn = "arn:aws:bedrock:us-east-1:942286715197:application-inference-profile/4eix5vfvs8bmm"
    
    # Create custom embedding model
    embed_model = BedrockProfileEmbedding(profile_arn)
    
    # Create sample documents
    documents = [
        Document(text="This is a document about artificial intelligence."),
        Document(text="Embeddings are vector representations of text."),
        Document(text="LlamaIndex helps with RAG applications.")
    ]
    
    # Create vector index using our Bedrock embeddings
    index = VectorStoreIndex.from_documents(
        documents,
        embed_model=embed_model
    )
    
    # Create a query engine
    query_engine = index.as_query_engine()
    
    # Execute a query
    response = query_engine.query("What are embeddings?")
    
    print("Query response:")
    print(response)
    
    return index

# Run the example
if __name__ == "__main__":
    print("Starting document indexing with Bedrock application inference profile...")
    index = index_documents_with_bedrock()
    print("Indexing complete!")

## APPROACH 8

In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to generate an embedding with the Amazon Titan Embeddings G1 - Text model (on demand).
"""

import json
import logging
import boto3


from botocore.exceptions import ClientError


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def generate_embedding(model_id, body):
    """
    Generate an embedding with the vector representation of a text input using Amazon Titan Embeddings G1 - Text on demand.
    Args:
        model_id (str): The model ID to use.
        body (str) : The request body to use.
    Returns:
        response (JSON): The embedding created by the model and the number of input tokens.
    """

    logger.info("Generating an embedding with Amazon Titan Embeddings G1 - Text model %s", model_id)

    bedrock = boto3.client(service_name='bedrock-runtime')

    accept = "application/json"
    content_type = "application/json"

    response = bedrock.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type
    )

    response_body = json.loads(response.get('body').read())

    return response_body


def main():
    """
    Entrypoint for Amazon Titan Embeddings G1 - Text example.
    """

    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")

    model_id = "amazon.titan-embed-text-v1"
    input_text = "What are the different services that you offer?"


    # Create request body.
    body = json.dumps({
        "inputText": input_text,
    })


    try:

        response = generate_embedding(model_id, body)

        print(f"Generated an embedding: {response['embedding']}")
        print(f"Input Token count:  {response['inputTextTokenCount']}")

    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
              format(message))

    else:
        print(f"Finished generating an embedding with Amazon Titan Embeddings G1 - Text model {model_id}.")


if __name__ == "__main__":
    main()


## APPROACH 9

In [ ]:
import boto3
from langchain.embeddings import BedrockEmbeddings

bedrock_client = boto3.client(service_name='bedrock-runtime', 
                              region_name='us-east-1')
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_client)

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
pdf_path = ""
loader = PyPDFLoader(pdf_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)


# faiss.write_index(vectorstore_faiss.index, "../../data/index/prompt_embeddings.index")
